This notebook will perform our temporal train/test split of the dataset, after merging the two existing datasets into one and filtering out entries in the Disaster Declaration Summaries (DDS) that do not exist in the Mission Assignments (MA)

In [128]:
import pandas as pd

In [129]:
ma_filepath = 'mission_assignments.parquet'
dds_filepath = 'disaster_declaration_summaries.parquet'
train_filepath = 'combined_training_set.parquet'
test_filepath = 'combined_test_set.parquet'


In [130]:
# Load initial datasets

df_dds = pd.read_parquet(dds_filepath)
df_ma = pd.read_parquet(ma_filepath)
print(df_dds.shape, df_ma.shape)

(68485, 28) (40340, 39)


Data cleaning for MA includes 

In [131]:
df_ma=df_ma[(df_ma['declarationType']!='SU')&(df_ma['maAmendNumber']==0)&(df_ma['supportFunction']<=15)]

df_ma['stt'].replace({'VA':'VA_state'},inplace=True)

df_ma['agencyId'].replace({'CISA':'DHS-CISA','DHSMGMT':'DHS-MGMT','USDANRCS':'USDA-NRCS','GSA-':'GSA','VA-':'VA','EPA-':'EPA','DOT-':'DOT',
                               'CNCS-':'CNCS','FCC-':'FCC','DOED':'DOE','DHUD':'HUD','DOD-':'DOD','VA -':'VA','USDAOCIO':'USDA-OCIO','FPS':'DHS-FPS',
                               'TSA':'DHS-TSA','ICE':'DHS-ICE','USCIS':'DHS-CIS','DLA':'DOD-DLA','CBP':'DHS-CBP','NPS':'DOI-NPS','NPPD':'DHS-CISA',
                              'CDC':'HHS-CDC','USAF':'DOD-USAF','OSHA':'DOL-OSHA','DHS-MGT':'DHS-MGMT','USGS':'DOI-USGS','USCG':'DHS-USCG',
                              'USDJ':'DOJ','DHS-MGA':'DHS-IA','FLETC':'DHS-FLETC','DHS-FLET':'DHS-FLETC','USFS':'USDA-FS','HHS -PSC':'HHS-PSC'},inplace=True)

column_list_ma = ['incidentId','stt','incidentType','region','maType','maPriority','supportFunction','agencyId', 'maId',
              'declarationType', 'assistanceRequested', 'statementOfWork']
df_ma = df_ma.reindex(columns=column_list_ma)

df_ma.drop_duplicates(inplace=True)

# df_ma.drop(columns=[
#     'lastRefresh',
#     'hash',
#     'id',],
#     inplace=True)

df_ma.shape

/var/folders/15/mh2mgzhn1191h4zq_5d03wfc0000gn/T/ipykernel_90853/1256216817.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_ma['stt'].replace({'VA':'VA_state'},inplace=True)
/var/folders/15/mh2mgzhn1191h4zq_5d03wfc0000gn/T/ipykernel_90853/1256216817.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values 

(7044, 12)

Data cleaning for DDS includes keeping of specific columns and filtering of year and declaration type

In [132]:
# select columns necessary for data analysis, add empty columns for each natural disaster type

column_list_dds = ['femaDeclarationString','state','incidentType','incidentBeginDate','fipsStateCode','region',
               'designatedIncidentTypes','declarationTitle', 'incidentId','declarationType']
df_dds = df_dds.reindex(columns=column_list_dds)

# Add time information to DDS

df_dds['incidentBeginDate']=pd.to_datetime(df_dds['incidentBeginDate'])
df_dds['year'] = df_dds['incidentBeginDate'].dt.year
df_dds['month'] = df_dds['incidentBeginDate'].dt.month
df_dds['day'] = df_dds['incidentBeginDate'].dt.day

# Filter out values before 2012

df_dds=df_dds[(df_dds['year']>=2012) & (df_dds['declarationType']!='FM')]
# df_dds.drop(columns=[
#     'lastRefresh',
#     'hash',
#     'id',],
#     inplace=True)
df_dds.drop_duplicates(inplace=True)
df_dds.reset_index(inplace = True)

df_dds.shape


(1123, 14)

In [133]:
dds_column_list = df_dds.columns.to_list()
dds_column_list

['index',
 'femaDeclarationString',
 'state',
 'incidentType',
 'incidentBeginDate',
 'fipsStateCode',
 'region',
 'designatedIncidentTypes',
 'declarationTitle',
 'incidentId',
 'declarationType',
 'year',
 'month',
 'day']

In [134]:
df_ma.rename(columns={'stt':'state'},inplace=True)
ma_column_list = df_ma.columns.to_list()
ma_column_list

['incidentId',
 'state',
 'incidentType',
 'region',
 'maType',
 'maPriority',
 'supportFunction',
 'agencyId',
 'maId',
 'declarationType',
 'assistanceRequested',
 'statementOfWork']

In [135]:
print(df_dds['incidentId'].nunique(), df_ma['incidentId'].nunique())

662 326


In [136]:
overlapping_columns = list(set(ma_column_list).intersection(set(dds_column_list)))
overlapping_columns

['incidentType', 'state', 'region', 'declarationType', 'incidentId']

In [ ]:
MA_disaster_combined=df_ma.merge(
    df_dds, 
    how='left',
    on=overlapping_columns,
    validate='m:m')

In [138]:
MA_disaster_combined.shape

(7635, 21)

In [139]:
MA_disaster_combined.head()

,incidentId,state,incidentType,region,maType,maPriority,supportFunction,agencyId,maId,declarationType,...,statementOfWork,index,femaDeclarationString,incidentBeginDate,fipsStateCode,designatedIncidentTypes,declarationTitle,year,month,day
0,2024081901,CT,Severe Storm,1,FOS,High,5.0,DOI-USGS,3612EMCTDOI-USGS01,EM,...,"As directed by and in coordination with FEMA, ...",64170.0,EM-3612-CT,2024-08-18 00:00:00+00:00,9.0,"M,W,F","SEVERE STORMS, FLOODING, LANDSLIDES, AND MUDSL...",2024.0,8.0,18.0
1,2024072801,FL,Tropical Storm,4,FOS,Normal,8.0,VA,4806DRFLVA02,DR,...,As directed by and in coordination with FEMA a...,15310.0,DR-4806-FL,2024-08-01 00:00:00+00:00,12.0,"4,H",HURRICANE DEBBY,2024.0,8.0,1.0
2,2024072801,FL,Tropical Storm,4,FOS,Normal,0.0,VA,4806DRFLVA01,DR,...,"As directed by and in coordination with FEMA, ...",15310.0,DR-4806-FL,2024-08-01 00:00:00+00:00,12.0,"4,H",HURRICANE DEBBY,2024.0,8.0,1.0
3,2024072801,FL,Tropical Storm,4,FOS,Normal,4.0,USDA-FS,4806DRFLUSDA-FS01,DR,...,"As directed by and in coordination with FEMA, ...",15310.0,DR-4806-FL,2024-08-01 00:00:00+00:00,12.0,"4,H",HURRICANE DEBBY,2024.0,8.0,1.0
4,2024072801,FL,Tropical Storm,4,FOS,Normal,11.0,USDA-APH,4806DRFLUSDA-APH01,DR,...,"As directed by and in coordination with FEMA, ...",15310.0,DR-4806-FL,2024-08-01 00:00:00+00:00,12.0,"4,H",HURRICANE DEBBY,2024.0,8.0,1.0


In [140]:
MA_disaster_combined.isna().sum()

incidentId                    0
state                         0
incidentType                  0
region                        0
maType                        0
maPriority                    0
supportFunction               0
agencyId                      0
maId                          0
declarationType               0
assistanceRequested           0
statementOfWork               0
index                       693
femaDeclarationString       693
incidentBeginDate           693
fipsStateCode               693
designatedIncidentTypes    6510
declarationTitle            693
year                        693
month                       693
day                         693
dtype: int64

In [141]:
MA_disaster_combined['incidentId'].nunique()

326

In [142]:
MA_disaster_combined[MA_disaster_combined['year']<2023].to_parquet(train_filepath)
MA_disaster_combined[MA_disaster_combined['year']>=2023].to_parquet(test_filepath)